#### SP Cashflow Modeling Presentation
---
#### Contents
- basic installment collateral cashflow forecast
- pass through
- warehouse structuring, scenario analysis, comprehensive analytics visuals
- securitization financing structuring, scenario analysis, comprehensive analytics visuals


In [1]:
from AssetModeling.Asset import Asset
from AssetModeling import AssetStore
from StructureModeling import StructureStore
from DealManager.DealManager import DealManager
from Utils.SPCFUtils import SPCFUtils
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

#### Single collateral modeling. show curves, metrics, yield table

In [2]:
asset1 = Asset(**AssetStore.ASSETSTORE['consumerLoan70'])

In [ ]:
# asset1.getStaticMetrics() # show static metric
# asset1.getDynamicMetrics(['cdrCurve','ltlCurve']) #show curves
# asset1.calculateYieldTable(pxList = range(90,100)) #show yield talbe

#### ramp up portfolio and showing balance sheet build up and analytics

In [3]:
asset1.addRampSchedule(rampSchedule = "20 for 12", px = "100 for 6 98")

In [4]:
print(asset1.getStaticMetrics(ramp=True)) # show static metrices

# show curves
'''
figPageRamp, axPageRamp = plt.subplots(1,3, figsize=(10,4))
axPageRamp = axPageRamp.flatten()
for i, metrics in enumerate(['periodicNetCF','cumulativeCF','portfolioBalance']):
    df = asset1.getDynamicMetrics(metrics = metrics,ramp=True)
    if metrics == "periodicNetCF":
        df.plot.area(x = "period", ax = axPageRamp[i], grid = True, title=metrics, stacked = False)
    else:
        df.plot(x = "period", ax = axPageRamp[i], grid = True, title=metrics)

plt.tight_layout();plt.show()
'''

                 matrics         value
0         purchasePeriod  1.200000e+01
1   totalPurchaseBalance  2.400000e+08
2     totalPurchaseBasis  2.376000e+08
3          avgPurchasePx  9.900000e+01
4            peakBalance  2.175494e+08
5      peakBalancePeriod  1.100000e+01
6                    wal  3.277241e+00
7        breakevenPeriod  5.000000e+01
8               totalPnL  8.160371e+07
9                  yield  1.285165e-01
10                  moic  1.343450e+00


'\nfigPageRamp, axPageRamp = plt.subplots(1,3, figsize=(10,4))\naxPageRamp = axPageRamp.flatten()\nfor i, metrics in enumerate([\'periodicNetCF\',\'cumulativeCF\',\'portfolioBalance\']):\n    df = asset1.getDynamicMetrics(metrics = metrics,ramp=True)\n    if metrics == "periodicNetCF":\n        df.plot.area(x = "period", ax = axPageRamp[i], grid = True, title=metrics, stacked = False)\n    else:\n        df.plot(x = "period", ax = axPageRamp[i], grid = True, title=metrics)\n\nplt.tight_layout();plt.show()\n'

#### Collateral scenarios. Static metrices and curves.

In [5]:
deal1 = DealManager(dealName = "plainCollateralDeal",
                    dealDescriptive = {"dealSubsector":"consumer loan",
                                       "dealSector":"Consumer",
                                       "assetOriginator":"SuperLoan"                                       
                        },
                    dealMisc = {"upfrontFees": 1e6},
                    assetScenarios = {"base": asset1},                    
                    )
deal1.copyAssetWithNewAssumption("base", "frontLoad", {"defaultTimingCurve": SPCFUtils.timingCurveParse("32/32/15", term = TERM)})
deal1.copyAssetWithNewAssumption("base", "backLoad", {"defaultTimingCurve": SPCFUtils.timingCurveParse("10/10/10", term = TERM)})


In [ ]:
# print(deal1.getAssetStaticMetrics(pxList = range(100, 89, -1))) # show static metrices
# show curves
# figPageCollatCurve, axPageCollatCurve = plt.subplots(2,3, figsize=(10,8))
# axPageCollatCurve = axPageCollatCurve.flatten()
# for i, metrics in enumerate(['cdrCurve','cprCurve','dqCurve', 'factorCurve', 'cnlCurve', 'ltlCurve']):
#     deal1.getAssetDynamicMetrics(metrics).plot(x = "period", ax = axPageCollatCurve[i], grid = True, title=metrics)

# plt.tight_layout();plt.show()


#### Pass-through (0 adv rate ABS structure). Bake in deal fees, and show economics under different assumption

In [ ]:


FinancingTerm = StructureStore.STRUCTURESTORE['PT']
deal2 = DealManager(dealName = "PTDeal",
                    dealDescriptive = {"dealSubsector":"subprime auto",
                                       "dealSector":"Consumer",
                                       "assetOriginator":"SuperLoan"                                       
                        },
                    dealMisc = {"upfrontFees": 1e6},
                    assetScenarios = {"base": AssetStore.ASSETSTORE['autoLoan45']},
                    financingTerms = FinancingTerm
                    )

deal2.copyAssetWithNewAssumption("base", "frontLoad", {"defaultTimingCurve": SPCFUtils.timingCurveParse("37/37/20", term = 45)})
deal2.copyAssetWithNewAssumption("base", "backLoad", {"defaultTimingCurve": SPCFUtils.timingCurveParse("10/10/10", term = 45)})

print(deal2.getAssetStaticMetrics())
print("*" * 50)
print(deal2.getStructureStaticMetrics())


#### A/B cut. (70/30 ABS Structure). single tranche term ABS.

In [ ]:


dealABCut = DealManager(dealName = "ABCutonConsumerLoan",
                    dealDescriptive = {"dealSubsector":"consumer loan",
                                       "dealSector":"Consumer",
                                       "assetOriginator":"SuperLoan"                                       
                        },
                    dealMisc = {"upfrontFees": 1e6},
                    assetScenarios = {"base": AssetStore.ASSETSTORE['consumerLoan58']},
                    financingTerms = StructureStore.STRUCTURESTORE['ABCut']
                    )

dealABCut.copyAssetWithNewAssumption("base", "frontLoad", {"defaultTimingCurve": SPCFUtils.timingCurveParse("50/35/10", term = 58)})
dealABCut.copyAssetWithNewAssumption("base", "backLoad", {"defaultTimingCurve": SPCFUtils.timingCurveParse("10/10/10", term = 58)})
dealABCut.addSeriesDefaultScenario(startingMultiple=1.0, endingMultiple=2.5, step=0.25)

print(dealABCut.getCapitalStack())
print("*" * 100)
print(dealABCut.getAssetStaticMetrics(pxList = range(100, 89, -1)))
print("*" * 100)
print(dealABCut.getStructureStaticMetrics())
print("*" * 100)
print(dealABCut.getAssetStaticMetrics(ramp = True))

# figPageCollat, axPageCollat = plt.subplots(2,3, figsize=(10,4))
# axPageCollat = axPageCollat.flatten()
# for i, metrics in enumerate(['cdrCurve','cprCurve','sevCurve','dqCurve','cnlCurve','factorCurve']):
#     df = dealABCut.getAssetDynamicMetrics(metrics)
#     df.plot(x = "period", ax = axPageCollat[i], grid = True, title=metrics)
# plt.tight_layout();plt.show()


#### securitization financing structuring, scenario analysis, comprehensive analytics visuals

#### warehouse structuring, scenario analysis, comprehensive analytics visuals